# Modelo de ejemplo

En esta notebook vamos a armar un ejemplo de como construir un modelo de punta a punta aplicando las ideas que vimos sobre el final del curso de pipelines y deploy. Nos vamos a focalizar el esos aspectos y no en los datos a analizar que van a ser de juguete.

In [156]:
# Vamos a leer un dataset y chequear que la info este bien, y luego separa en test/train

import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv('pokemon.csv')
X_train, X_test, y_train, y_test = train_test_split(data.drop(columns = ['fuerza_combate']), data['fuerza_combate'], test_size=0.33,random_state = 127)
X_train

,nombre,indice_guia,resistencia,ataque,defensa,tipo_primario,tipo_secundario,max_salud,tasa_captura,tasa_escape,Weight,Height,Legendario,generacion
506,Herdier,507,163.0,145.0,126.0,Normal,None,140.0,0.25,0.09,14.7,0.90,No,5
561,Yamask,562,116.0,95.0,141.0,Ghost,None,103.0,0.30,0.10,1.5,0.50,No,5
636,Volcarona,637,198.0,264.0,189.0,Bug,Fire,168.0,0.10,0.07,46.0,1.60,No,5
209,Granbull,210,207.0,212.0,131.0,Fairy,None,175.0,0.15,0.08,48.7,1.40,No,2
19,Raticate,20,146.0,161.0,139.0,Normal,None,127.0,0.20,0.07,18.5,0.70,No,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,Torkoal,324,172.0,151.0,203.0,Fire,None,147.0,0.30,0.09,80.4,0.51,No,3
36,Vulpix,37,116.0,96.0,109.0,Fire,None,103.0,0.30,0.10,9.9,0.60,No,1
92,Haunter,93,128.0,223.0,107.0,Ghost,Poison,113.0,0.20,0.07,0.1,1.60,No,1
60,Poliwhirl,61,163.0,130.0,123.0,Water,None,140.0,0.25,0.07,20.0,1.00,No,1


# Vamos ahora a pensar los pasos que queremos hacer:

## Limpieza de datos (la parte de visualizacion y analisis la salteamos porque no es el objetivo del ejemplo)

- Hacer algo con los nans: opciones: tirar filas, imputar por valor promedio, imputar por valor random
- Procesar columnas no numericas: opciones: hacer dummy, transformar en escala, tirar
- Escalado

## Modelos

- Queremos probar con un knn y con un random forest

Se podrian aplicar muchos mas criterios pero no lo hacemos porque es solo un ejemplo de como implementar-

# Preparamos los steps

In [159]:
# Clase que hace el fit-transform de los nans

from sklearn.base import BaseEstimator, TransformerMixin

class procesar_nans(BaseEstimator, TransformerMixin):
    """ Clase que preprocesa los datos para hacer algo con los nans, los tres criterios validos son: 'mean','sample','none'.
    Drop no es una opcion porque se pierde la coherencia de filas entre train y test.
     """
    def __init__(self,criterio='mean'):
        if not criterio in ['mean','sample','none']:
            raise ValueError('criterio no valido')
        self.criterio = criterio
        self.promedios = None
    
    def fit(self,X,y=None):
        if self.criterio == 'mean': # El unico caso en que tiene sentido buscar un valor en train es cuando se busca un valor promedio.
            self.promedios = {}
            for col in X.columns:
                self.promedios[col] = X[col].mean()
        return self
    
    def transform(self, X, y=None):
        if self.criterio == 'mean':
            for col in X.columns:
                X[col].fillna(self.promedios[col],inplace=True)
        if self.criterio == 'sample':
            for col in X.columns:
                mascara = X[col].isna()
                X.loc[mascara,col] = X.loc[~ mascara,col].sample(mascara.sum()).values
        if self.criterio == 'none':
            pass
        return X

In [124]:
# Clase que hace el fit-transform de las columnas no numericas

from sklearn.base import BaseEstimator, TransformerMixin

class procesar_not_numeric(BaseEstimator, TransformerMixin):
    """ Clase que preprocesa los datos para hacer algo con las columnas no numericas, los cuatro criterios validos son: 'drop','dummy','order','none' 
    En caso de especificar order se debe incluir un diccionario con los ordenes. 
    Se puede especificar un limite de categorias para eliminar las columnas que tengan mas categorias que ese limite.
    Se puede especificar que si una columna tiene un campo que sea None se binarice dicha columna en si hay valor o no.
    Se puede especificar a que columnas se quiere aplicar el procesamiento. """
    def __init__(self,criterio='drop',order=None,limite_categorias=None,binarizar_si_es_none=None,columns=None):
        if not criterio in ['drop','dummy','order','none']:
            raise ValueError('criterio no valido')
        if criterio == 'order' and order == None:
            raise ValueError('Para ordenar se debe incluir el parametro que lo establece.')
        self.criterio = criterio
        self.order = order
        self.limite_categorias = limite_categorias
        self.binarizar_si_es_none = binarizar_si_es_none
        self.columns = columns
    
    def fit(self,X,y=None):
        # Completamos con las columnas a procesar si no se especificaron
        if not self.columns:
            self.columns = X.columns
        # Chequeamos que el order tenga todo lo necesario dado el dataframe con que se entrena
        if self.criterio == 'order':
            for col in self.columns:
                if X[col].dtype == 'object':
                    if not col in self.order.keys():
                        raise ValueError(f'No se encontro la columna {col} en el diccionario que establece los ordenes')
        return self
    
    def transform(self, X, y=None):
        if self.limite_categorias:
            for col in self.columns:
                if X[col].dtype == 'object':
                    if X[col].nunique() > self.limite_categorias:
                        X = X.drop(columns = col)
        if self.binarizar_si_es_none:
            for col in self.columns:
                if X[col].dtype == 'object':
                    X[col] = X[col] == "None"
        if self.criterio == 'drop':
            X = X._get_numeric_data()
        if self.criterio == 'dummy':
            X = pd.get_dummies(X, drop_first=True)
        if self.criterio == 'order':
            for col in self.columns:
                if X[col].dtype == 'object':
                    X[col] = X[col].map(self.order[col], na_action='ignore')
        if self.criterio == 'none':
            pass
        return X

In [119]:
X_train, X_test, y_train, y_test = train_test_split(data.drop(columns = ['fuerza_combate']), data['fuerza_combate'], test_size=0.33,random_state = 127)

paso2= procesar_not_numeric('dummy', limite_categorias=20, binarizar_si_es_none=False, columns=['tipo_secundario','tipo_primario'])

data_t = paso2.fit_transform(X_train)
display (data_t)


,indice_guia,resistencia,ataque,defensa,max_salud,tasa_captura,tasa_escape,Weight,Height,generacion,...,tipo_secundario_Grass,tipo_secundario_Ground,tipo_secundario_Ice,tipo_secundario_None,tipo_secundario_Poison,tipo_secundario_Psychic,tipo_secundario_Rock,tipo_secundario_Steel,tipo_secundario_Water,Legendario_Sí
506,507,163.0,145.0,126.0,140.0,0.25,0.09,14.7,0.90,5,...,0,0,0,1,0,0,0,0,0,0
561,562,116.0,95.0,141.0,103.0,0.30,0.10,1.5,0.50,5,...,0,0,0,1,0,0,0,0,0,0
636,637,198.0,264.0,189.0,168.0,0.10,0.07,46.0,1.60,5,...,0,0,0,0,0,0,0,0,0,0
209,210,207.0,212.0,131.0,175.0,0.15,0.08,48.7,1.40,2,...,0,0,0,1,0,0,0,0,0,0
19,20,146.0,161.0,139.0,127.0,0.20,0.07,18.5,0.70,1,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,324,172.0,151.0,203.0,147.0,0.30,0.09,80.4,0.51,3,...,0,0,0,1,0,0,0,0,0,0
36,37,116.0,96.0,109.0,103.0,0.30,0.10,9.9,0.60,1,...,0,0,0,1,0,0,0,0,0,0
92,93,128.0,223.0,107.0,113.0,0.20,0.07,0.1,1.60,1,...,0,0,0,0,1,0,0,0,0,0
60,61,163.0,130.0,123.0,140.0,0.25,0.07,20.0,1.00,1,...,0,0,0,1,0,0,0,0,0,0


In [167]:
# Vamos a importar todas las cosas que nos faltan y vamos a armar el pipeline con un modelo simple

from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, StratifiedKFold 


In [168]:
pasos = [('limpieza_nans',procesar_nans()),('categoricas1',procesar_not_numeric()),('categoricas2',procesar_not_numeric()),('scaler', StandardScaler()),('model', LinearRegression())]
pipe = Pipeline(pasos)

In [135]:
#pipe.get_params()

In [169]:
# Cargamos alguna configuracion basica y probamos

X_train, X_test, y_train, y_test = train_test_split(data.drop(columns = ['fuerza_combate','nombre','indice_guia','tipo_primario','tipo_secundario','Legendario']), data['fuerza_combate'], test_size=0.33)
display (y_train)
pasos = [('limpieza_nans',procesar_nans()),('scaler', StandardScaler()),('modelo', LinearRegression())]
pipe = Pipeline(pasos)
param_grid = {'limpieza_nans__criterio':['mean','sample']}
folds=StratifiedKFold(n_splits=5,shuffle=True)
grid = GridSearchCV(pipe, param_grid, cv=folds)
grid.fit(X_train, y_train)


516    1145
108    1214
182     461
577    2018
616    2441
       ... 
171     473
357    2259
631    2659
496    2277
503     791
Name: fuerza_combate, Length: 434, dtype: int64

c:\Users\Ionatan\anaconda3\envs\DH\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=True),
             estimator=Pipeline(steps=[('limpieza_nans', procesar_nans()),
                                       ('scaler', StandardScaler()),
                                       ('modelo', LinearRegression())]),
             param_grid={'limpieza_nans__criterio': ['mean', 'sample']})

In [162]:
X_train, X_test, y_train, y_test = train_test_split(data.drop(columns = ['fuerza_combate','nombre','indice_guia','tipo_primario','tipo_secundario','Legendario']), data['fuerza_combate'], test_size=0.33)
paso1 = procesar_nans()
paso2 = StandardScaler()
X_train = paso1.fit_transform(X_train)
X_train = paso2.fit_transform(X_train)
folds=StratifiedKFold(n_splits=5,shuffle=True)
param_grid = {'n_neighbors':[5,10,50]}
grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=folds)
grid.fit(X_train, y_train)
display (X_train)

c:\Users\Ionatan\anaconda3\envs\DH\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


array([[-0.6140193 , -0.90938767, -0.83011954, ..., -0.61470768,
        -0.74960972,  0.61941017],
       [-0.91445657, -1.16066968, -1.44304865, ..., -0.53807545,
        -0.64830793,  1.28317316],
       [ 0.02977486,  1.08291968,  0.58595808, ..., -0.05193974,
         0.56731361, -0.04435283],
       ...,
       [-0.33504183, -1.12477225, -0.87239051, ..., -0.31775779,
        -0.14179895, -1.37187881],
       [-1.06467521, -1.16066968, -1.44304865, ..., -0.55603613,
        -0.74960972,  1.28317316],
       [ 0.26583272,  1.06497096,  0.20551932, ..., -0.01841314,
         0.06080464,  0.61941017]])